## beta Nitrogen

https://next-gen.materialsproject.org/materials/mp-672233?formula=N2

In [20]:
import numpy as np
import re

_nitrogens = """N1 1/3 2/3 0.32893
""".splitlines()

atomd = dict()
for line in _nitrogens:
    cols = line.split()
    row = []
    for x in cols[1:4]:
        m = re.search(f"^([0-9./]+)", x)
        row.append(eval(m.group(1)))
    atomd[cols[0]] = row

atomd

{'N1': [0.3333333333333333, 0.6666666666666666, 0.32893]}

In [21]:
from genice2.cell import cellvectors
from genice2 import CIF

a = 4.17 / 10.0  # nm
c = 7.01 / 10.0 # nm
A = 90
C = 120

# cellvectors関数は単位胞の幾何形状情報を行列にする。
cell = cellvectors(a, a, c, A, A, C)
cell

array([[ 0.417     ,  0.        ,  0.        ],
       [-0.2085    ,  0.36113259,  0.        ],
       [ 0.        ,  0.        ,  0.701     ]])

In [22]:
from logging import getLogger, basicConfig, INFO

basicConfig(level=INFO)

# space group P6₃/mmc の対称操作 (No. 194)
# http://img.chem.ucl.ac.uk/sgp/large/217az1.htm
symops = """   x,            y,            z
     -y,           x-y,           z
    -x+y,          -x,            z
     -x,           -y,          1/2+z
     x-y,           x,          1/2+z
      y,          -x+y,         1/2+z
     -y,           -x,            z
    -x+y,           y,            z
      x,           x-y,           z
      y,            x,          1/2+z
     x-y,          -y,          1/2+z
     -x,          -x+y,         1/2+z
     -x,           -y,           -z
      y,          -x+y,          -z
     x-y,           x,           -z
      x,            y,          1/2-z
    -x+y,          -x,          1/2-z
     -y,           x-y,         1/2-z
      y,            x,           -z
     x-y,          -y,           -z
     -x,          -x+y,          -z
     -y,           -x,          1/2-z
    -x+y,           y,          1/2-z
      x,           x-y,         1/2-z
"""


# 対称操作表に従い、原子位置を複製する。すべての原子位置がこれで求められる。
atoms = CIF.fullatoms(atomd, sops=CIF.symmetry_operators(symops))
atoms

[('N1', array([0.33333333, 0.66666667, 0.32893   ])),
 ('N1', array([-0.33333333, -0.66666667,  0.82893   ])),
 ('N1', array([-0.33333333, -0.66666667, -0.32893   ])),
 ('N1', array([0.33333333, 0.66666667, 0.17107   ]))]

In [23]:
len(atoms)

4

In [24]:
nitrogens = np.array([atom[1] for atom in atoms if atom[0][0] == "N"])
nitrogens -= np.floor(nitrogens+0.5)

nitrogens @ cell

array([[ 0.2085    , -0.12037753,  0.23057993],
       [-0.2085    ,  0.12037753, -0.11992007],
       [-0.2085    ,  0.12037753, -0.23057993],
       [ 0.2085    , -0.12037753,  0.11992007]])

In [25]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=nitrogens[:, 0],
        y=nitrogens[:, 1],
        z=nitrogens[:, 2],
        # x=vecs[zpos<0.5, 0],
        # y=vecs[zpos<0.5, 1],
        # z=vecs[zpos<0.5, 2],
        mode="markers",
        # marker=dict(size=2, color=zpos[:]),
    )
)

fig.update_layout(
    title="local arrangement",
    autosize=True,
    width=800,
    height=800,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()

groファイルを作ってみよう。

In [35]:
import gromacs

nitrogensx = []
# abc方向に242倍し、b軸をつけかえる。
for x in range(2):
    for y in range(4):
        for z in range(2):
            nitrogensx.append(nitrogens + np.array([x,y,z]))
# 座標を0〜1におさめる
nitrogensx = np.vstack(nitrogensx) / np.array([2,4,2])
# セルを大きくする。
cellx = cell * np.array([2,4,2])

abs_nitrogens = nitrogensx @ cellx

# 60度の傾きを補正
cellx[1,0] = 0
celli = np.linalg.inv(cellx)
abs_nitrogens = np.floor(abs_nitrogens @ celli) @ cellx

residues = dict()
residues["N2"] = [
    [
        ["N", pos],
    ]
    for pos in abs_nitrogens
]

frame = gromacs.compose(residues, cellx)
with open("NitrogenB.gro", "w") as f:
    gromacs.write_gro(frame, f)
cellx

array([[0.834     , 0.        , 0.        ],
       [0.        , 1.44453037, 0.        ],
       [0.        , 0.        , 1.402     ]])